In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# @vla: custom parse options and quantize function
from lavis.args_parser import *
from lavis.quantize import quantize, model_size

import lavis.tasks as tasks
from lavis.common.config import Config

from lavis.layers.nbitlineardynamic import NBitLinearDynamic

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
parser = args_parser()

In [11]:
CLI_INPUT = f'''
                 --cfg-path ret_flickr_eval.yaml
                 --visual-encoder-block-indices {','.join([str(i) for i in range(39)])}
                 --visual-encoder-block-modules qkv,proj,fc1,fc2
                 --visual-encoder-block-weight-bits -1
                 --qformer-layer-indices {','.join([str(i) for i in range(12)])}
                 --qformer-self-attention-modules query,key,value,dense
                 --qformer-self-attention-weight-bits -1
                 --qformer-cross-attention-modules query,key,value,dense
                 --qformer-cross-attention-weight-bits -1
                 --qformer-text-ff-modules intermediate,output
                 --qformer-text-ff-weight-bits -1
                 --qformer-img-ff-modules intermediate_query,output_query
                 --qformer-img-ff-weight-bits -1
                 --qformer-cls-modules transform,decoder
                 --qformer-cls-transform-weight-bits -1
                 --qformer-cls-decoder-weight-bits -1
                 --output-modules vision_proj,text_proj,itm_head
                 --vision-proj-weight-bits -1
                 --text-proj-weight-bits -1
                 --itm-head-weight-bits -1
                 --default-weight-bits 8 
             '''
             
             
# CLI_INPUT = f'''--cfg-path ret_flickr_eval.yaml --visual-encoder-block-modules qkv,proj,fc1,fc2'''

#  --visual-encoder-block-indices 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38 --visual-encoder-block-weight-bits 8'''

In [12]:
args = parser.parse_args(CLI_INPUT.split())
args = validate_args(args)
vars(args)

{'cfg_path': 'ret_flickr_eval.yaml',
 'options': None,
 'visual_encoder_block_modules': ['qkv', 'proj', 'fc1', 'fc2'],
 'visual_encoder_block_indices': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38],
 'visual_encoder_block_weight_bits': 8,
 'qformer_layer_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'qformer_self_attention_modules': ['query', 'key', 'value', 'dense'],
 'qformer_self_attention_weight_bits': 8,
 'qformer_cross_attention_modules': ['query', 'key', 'value', 'dense'],
 'qformer_cross_attention_weight_bits': 8,
 'qformer_text_ff_modules': ['intermediate', 'output'],
 'qformer_text_ff_weight_bits': 8,
 'qformer_img_ff_modules': ['intermediate_query', 'output_query'],
 'qformer_img_ff_weight_bits': 8,
 'qformer_cls_modules': ['transform', 'decoder'],
 'qformer_cls_transform_weight_bits': 8,
 'qform

In [10]:
args_dict = vars(args)

for key in args_dict:
    if 'weight_bits' in key and args_dict[key] != None:
        setattr(args, key, 'test')

In [11]:
args

Namespace(cfg_path='ret_flickr_eval.yaml', options=None, visual_encoder_block_modules=['qkv', 'proj', 'fc1', 'fc2'], visual_encoder_block_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], visual_encoder_block_weight_bits='test', qformer_layer_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], qformer_self_attention_modules=['query', 'key', 'value', 'dense'], qformer_self_attention_weight_bits='test', qformer_cross_attention_modules=['query', 'key', 'value', 'dense'], qformer_cross_attention_weight_bits='test', qformer_text_ff_modules=['intermediate', 'output'], qformer_text_ff_weight_bits='test', qformer_img_ff_modules=['intermediate_query', 'output_query'], qformer_img_ff_weight_bits='test', qformer_cls_modules=['transform', 'decoder'], qformer_cls_transform_weight_bits='test', qformer_cls_decoder_weight_bits='test', output_modules=['vision_proj', 'text_proj', 'itm_head'], vision_pro

In [6]:
cfg = Config(args)
cfg

In [7]:
task = tasks.setup_task(cfg)
task

In [8]:
model = task.build_model(cfg)
model

MODEL_NAME: eva_clip_g
Position interpolate from 16x16 to 26x26


Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [11]:
size = 0

for param in model.parameters():
    size += param.nelement() * (param.element_size() * 8)

for buffer in model.buffers():
    size += buffer.nelement() * (buffer.element_size() * 8)
    
size /= (8e6)
size

4692.769528

In [12]:
model_size(model)

4782.180084

In [23]:
quantize(model, args)

In [24]:
model

Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): NBitLinearDynamic(in_features=1408, out_features=4224, bias=False | w=8, a=32)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): NBitLinearDynamic(in_features=1408, out_features=1408, bias=True | w=8, a=32)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): NBitLinearDynamic(in_features=1408, out_features=6144, bias=True | w=8, a=32)
          (act): GELU(approximate='none')
          (fc2): NBitLinearDynamic(in_features=6144, out_features=1408, bia

In [25]:
# quantized model size (MB)
model_size(model)

1216.125832875

In [45]:
class Argument(object):

    def __init__(self, name, cmd_line, string_id, val, multiarg=False):
        
        self.name = name
        self.val = val
        
        # support for assigning groups of args
        if isinstance(val, tuple):
            
            assert len(val) == len(cmd_line)
            
            # expecting val to be a tuple, where each arg of tuple is set to each each cli argument in cmd_line
            if isinstance(cmd_line, list):
                self.cmd_string = ''
                for cur_line,arg_val in zip(cmd_line, val):
                    
                    # False values --> skip argument
                    if arg_val == False:
                        continue
                    if isinstance(arg_val,list):
                        self.cmd_string += ' ' + cur_line+' '+','.join([str(e) for e in arg_val])
                    else:
                        self.cmd_string += ' ' + cur_line+' '+ str(arg_val)
            else:
                raise Exception('cmd_line must be list')
            
        else:
            if isinstance(val,list):
                if len(val) == 0:

                    if isinstance(cmd_line, list):
                        self.cmd_string = ''
                        for cur_line in cmd_line:
                            self.cmd_string += ' '+cur_line+' []'
                    else:
                        self.cmd_string = ' '+cmd_line+' []'
                else:
                    if isinstance(cmd_line, list):
                        self.cmd_string = ''
                        for cur_line in cmd_line:
                            self.cmd_string += ' '+cur_line+' '+','.join([str(e) for e in val])
                    else:
                        self.cmd_string = ' '+cmd_line+' '+','.join([str(e) for e in val])
            else:

                if isinstance(cmd_line, list):
                    self.cmd_string = ''
                    for cur_line in cmd_line:
                        self.cmd_string += ' '+cur_line+' '+str(val)
                else:
                    self.cmd_string = ' '+cmd_line+' '+str(val)
                    
        if isinstance(val,bool):
            
            if not val:
                self.job_string = ''
                self.cmd_string = ''
                self.name = ''
            else:
                self.job_string = '_'+string_id if string_id else ''
                if isinstance(cmd_line, list):
                    self.cmd_string = ''
                    for cur_line in cmd_line:
                        self.cmd_string += ' '+cur_line+' '
                self.cmd_string = ' '+cmd_line+' '
        elif isinstance(val,list):
            self.job_string = '_'+string_id+'_'.join([str(v) for v in val])
        else:
            self.job_string = '_'+string_id+str(val)
        if string_id == 'none':
            self.job_string = ''
        

    def copy(self):
        new_arg = Argument(self.name, cmd_line='', string_id='', val=self.val)
        new_arg.cmd_string = self.cmd_string
        new_arg.job_string = self.job_string
        return new_arg


In [51]:
params = {
    'cfg-path': ['--cfg-path', 'none', ['ret_flickr_eval.yaml']],              # set BLIP-2 model + task
    'weight_bits': [['--a', '--b', '--c'],
                    'weight_bits_',
                    [(1,2,3), (False, 4,5), (5,6, False)]]
}


In [53]:
cur_arg_list = []

for key, param in params.items():
    for value in param[2]:
        print(value)
        cur_arg_list.append(Argument(key, param[0],param[1], value))

ret_flickr_eval.yaml
(1, 2, 3)
(False, 4, 5)
(5, 6, False)


In [55]:
cur_arg_list[3].cmd_string

' --a 5 --b 6'

In [56]:
cur_arg_list[3].job_string

'_weight_bits_(5, 6, False)'

In [44]:
list(zip(['a', 'b', 'c'], (1,2,3)))

[('a', 1), ('b', 2), ('c', 3)]